<a href="https://colab.research.google.com/github/utsabdangol/machine_learing_python/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
#data
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])
label_map = {'ham': 0, 'spam': 1}
y_train = train_df['label'].map(label_map).values
y_test = test_df['label'].map(label_map).values

# Define text vectorization layer
max_tokens = 10000
max_len = 100

vectorizer = keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode='int',
    output_sequence_length=max_len
)

# Adapt to training messages
vectorizer.adapt(train_df['message'].values)
X_train = vectorizer(train_df['message'].values)
X_test = vectorizer(test_df['message'].values)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
model = keras.Sequential([
    keras.layers.Embedding(max_tokens, 32, input_length=max_len),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')  # important sigmoid here
])
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
early_stop = EarlyStopping(
    monitor='val_loss',      # watch the validation loss
    patience=2,              # stop if val_loss doesn't improve for 2 consecutive epochs
    restore_best_weights=True  # restore weights from the best epoch
)
model.build(input_shape=(None, 250))
model.summary()
history = model.fit(
    X_train, y_train,
    epochs=20,
    validation_data=(X_test, y_test),
    batch_size=32,
    callbacks=[early_stop]
)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

def predict_message(pred_text):
    # Convert and vectorize the input message
    message_tensor = tf.convert_to_tensor([pred_text])
    vectorized_message = vectorizer(message_tensor)

    # Predict using the trained model
    prediction = model.predict(vectorized_message)[0][0]

    # Choose label based on threshold (0.5)
    label = "spam" if prediction > 0.5 else "ham"

    # Return both the prediction score and the label
    return [prediction, label]


pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      print(msg)
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
